In [8]:
import math
import datetime, time
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn

In [9]:
def getMSH (ticker, strt, endt, intrvl):
    format = '%Y-%m-%d'
    dtstrt = datetime.datetime.strptime(strt, format)
    dtendt = datetime.datetime.strptime(endt, format)
    time_delta = dtendt-dtstrt
    time_marshall_as_year = time_delta.total_seconds() / 365.25 / 86400
    print("year frac = ", time_marshall_as_year)
    data = yf.download(ticker, start=strt, end=endt, interval=intrvl)
    tau = time_marshall_as_year / len(data['Adj Close'])
    ticks = np.arange(1,len(data['Adj Close'])+1)
    np_data = data['Adj Close']
    log_data = np.zeros(len(np_data))
    div = np_data[0]
    log_data = np.log(np_data / div)
    XT = log_data[len(log_data)-1]
    mean = XT / len(log_data)
    logs = log(np_data[:-1]/np_data[1:])
    groups = np.arange(int(len(logs) / 10), 1, -1)
    RS = np.zeros(len(groups)+1)
    line = np.zeros(len(groups)+1)
    
    for group in groups:
        sub_RS = np.zeros(group)
        stdev = np.zeros(group)
        line[group-1] = len(logs)/group
        for sub in range(group):
            RSmax, RSmin = 0, 0
            sub_group = np.array(logs[int(len(logs)/group)*sub : int(len(logs)/group*(sub+1))])
            for in_sub in range(1, int(len(logs)/group)):
                sub_RS[sub] += sub_group[in_sub]
                RSmax = max(sub_RS[sub], RSmax)
                RSmin = min(sub_RS[sub], RSmin)
            sub_RS[sub] = RSmax - RSmin
            stdev[sub] = std(sub_group)
        a = sub_RS / stdev
        RS[group-1] = a.mean()
    poly = polyfit(log(line[1:]), log(RS[1:]), 1)
    H = poly[0]
    
    var = 0
    for i in range(1, len(log_data)):
        var += (log_data[i] - log_data[i-1]) * (log_data[i] - log_data[i-1])
    var /= len(log_data) * tau
    mu = XT/time_marshall_as_year
    sigma = math.sqrt(var)
    hrst = hurst(data['Adj Close'])
    #print("mu = ", mu)
    #print("sigma = ", sigma)
    
    return mu, sigma, H

In [10]:
ticker = "JNJ"
strt = "2020-01-01"
endt = "2021-01-01"
intrvl = "1d"

st_y = ['2020-','2019-','2018-','2017-','2016-','2015-','2014-','2013-','2012-','2011-','2010-']
st_m = ['12-','11-','10-','09-','08-','07-','06-','05-','04-','03-','02-','01-']

para = np.zeros((len(st_y)*len(st_m),3), dtype="float32")
i = 0

for sy in st_y:
    for sm in st_m:
        dts = sy + sm + "01"
        print (dts)
        para[i] = getMSH(ticker, dts, endt, intrvl)
        i += 1
    
print(para)

plt.plot(np.arange(i)+1, para[:,0])
plt.show()

plt.plot(np.arange(i)+1, para[:,1])
plt.show()

plt.plot(np.arange(i)+1, para[:,2])
plt.show()

2020-12-01
year frac =  0.08487337440109514
[*********************100%***********************]  1 of 1 completed


LinAlgError: SVD did not converge in Linear Least Squares